In [1]:


def get_json(response):
    # Define the regular expression for categories from the main page
    pattern = re.compile(r'window\.yelp\.react_root_props\s*=\s*(\{.*?\});')

    # Search for the pattern in the input string
    match = pattern.search(response.text)

    # Extract the JSON string
    if match:
        json_str = match.group(1)

        # Load the JSON data
        j = json.loads(json_str)

        # Print or process the JSON data
    else:
        print("No match found")
    return j


def write_json(data, filename):
    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Data successfully written to {filename}")
    except Exception as e:
        print(f"An error occurred: {e}")
        
        
def find_key_path(json_obj, target_key, current_path=""):
    """
    Finds the path to the target key in a nested JSON object.

    Parameters:
    json_obj (dict or list): The JSON object to search.
    target_key (str): The key to find.
    current_path (str): The current path being explored (used in recursion).

    Returns:
    list: A list of paths where the target key is found.
    """
    paths = []

    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            new_path = f"{current_path}/{key}" if current_path else key
            if key == target_key:
                paths.append(new_path)
            paths.extend(find_key_path(value, target_key, new_path))
    elif isinstance(json_obj, list):
        for index, item in enumerate(json_obj):
            new_path = f"{current_path}[{index}]"
            paths.extend(find_key_path(item, target_key, new_path))

    return paths

In [2]:
import pandas as pd
import re
import requests
import json
from itertools import cycle
from bs4 import BeautifulSoup

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}
home_page = "https://www.yelp.com/"
response = requests.get(home_page, headers=headers)

home_page_json = get_json(response)


In [3]:


initial_cat = home_page_json["legacyProps"]["category_tiles_props"]["initialCategories"]

all_cat = []
for i in initial_cat:
    all_cat.append(home_page+i['category']['url'])
    

more_cat = home_page_json["legacyProps"]["category_tiles_props"]["moreCategories"]
for i in more_cat:
    all_cat.append(home_page+i['category']['url'])

In [4]:
dic = {
    'name':[],
    'address':[],
    'tele_num':[],
#     'latitude':[],
#     'longitude':[],
    'link':[],
    'mainweb_link':[],
    'pagination_link':[],
    'category':[]
}

In [5]:

for cat_url in all_cat[:3]:
    ct = 0
    for num_page in range(5):
        category_url = cat_url+ f'&start={ct}'
        pagination_link = category_url
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
                }
        response = requests.get(category_url, headers=headers)
        cat_page_json = get_json(response)
        
        
        sub_links = cat_page_json["legacyProps"]["searchAppProps"]["searchPageProps"]["mainContentComponentsListProps"]
        location_axis = cat_page_json["legacyProps"]["searchAppProps"]["searchPageProps"]["rightRailProps"]["searchMapProps"]["mapState"]["markers"]
        
        for i in sub_links:
            if "searchResultBusiness" in i and i["searchResultBusiness"]["isAd"] == False:
                per_hotellinks = 'https://www.yelp.com/'+ i["searchResultBusiness"]["businessUrl"]
                print(per_hotellinks)
                headers = {
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                        'Accept-Language': 'en-US,en;q=0.9',
                        'Accept-Encoding': 'gzip, deflate, br',
                        'Connection': 'keep-alive',
                        'Upgrade-Insecure-Requests': '1'
                    }
                response = requests.get(per_hotellinks, headers=headers)
                per_hotellinks_json = get_json(response)
                category_ = per_hotellinks.split('=')[-1]
                print(pagination_link)
                print(category_)
                
                name = per_hotellinks_json["legacyProps"]["bizDetailsProps"]["bizDetailsPageProps"]["businessName"]
#               write_json(per_hotellinks_json, f'per_hotellinks_json{cout}.json')
                print(name)
                soup = BeautifulSoup(response.content,'html.parser')
                add_match = ''
                if soup.find('p',class_='y-css-dg8xxd'):
                    add_match = soup.find('p',class_='y-css-dg8xxd').text
                    print(add_match)
                else:
                    print("No address found.")
                    
        
                
#                 #Latitude
#                 latitude = cat_page_json["legacyProps"]["searchAppProps"]["searchPageProps"]["rightRailProps"]["searchMapProps"]["mapState"]["markers"][1]["location"]["latitude"]                   
#                 if latitude:
#                     print(f"Latitude = {lat}")
#                 else:
#                     print("No Latitude found.")
                
                
#                 #longitude
#                 longitude = cat_page_json["legacyProps"]["searchAppProps"]["searchPageProps"]["rightRailProps"]["searchMapProps"]["mapState"]["markers"][1]["location"]["longitude"]                   
#                 if longitude:
#                     print(f"Latitude = {longitude}")
#                 else:
#                     print("No Longitude found.")

                
                link_pattern = r'rel="noopener.*?">(.*?)</a></p>'
                # Search for the pattern in the HTML content
                link_match = re.search(link_pattern, response.text)
                # If a match is found, print the link text
                web_link = ''
                if link_match:
                    print(link_match.group(1))
                    web_link = link_match.group(1)
                else:
                    print("No link text found.")
                    
                
                
                tele_pattern = r'Phone number</p>.*?(\(.*?)</p>'
                # Search for the pattern in the HTML content
                tele_match = re.search(tele_pattern,response.text)
                # If a match is found, print the phone number
                web_num = ''
                if tele_match:
                    print(tele_match.group(1))
                    web_num = tele_match.group(1)
                else:
                    print("No phone number found.")
                 
                
                dic['name'].append(name)
                dic['address'].append(add_match)
                dic['tele_num'].append(web_num)
#                 dic['latitude'].append(latitude)
#                 dic['longitude'].append(longitude)
                dic['link'].append(web_link)
                dic['mainweb_link'].append(per_hotellinks)
                dic['pagination_link'].append(pagination_link)
                dic['category'].append(category_)
                
        ct+=10
        

https://www.yelp.com//biz/bueokae-korean-restaurant-loves-kitchen-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=0
Restaurants
Bueokae Korean Restaurant - Love's Kitchen
10737 Yonge St Unit 2/3 Richmond Hill, ON L4C 3E3 Canada
No link text found.
(905) 237-0737
https://www.yelp.com//biz/chillax-and-co-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=0
Restaurants
Chillax & Co
9580 Yonge Street Unit 101 Richmond Hill, ON L4C 1V6 Canada
chillaxandco.com
(905) 787-8666
https://www.yelp.com//biz/the-big-casserole-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=0
Restaurants
The Big Casserole
1480 Major MacKenzie Drive E Richmond Hill, ON L4S 0A1 Canada
No link text found.
(905) 508-3880
https://www.yelp.com//biz/jatujak-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_lo

360 Highway 7 Unit 5 Richmond Hill, ON L4B 3Y7 Canada
No link text found.
(905) 709-4660
https://www.yelp.com//biz/jin-korean-noodles-markham?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=30
Restaurants
Jin Korean Noodles
230 Commerce Valley Drive E Unit 7 Markham, ON L3T 7Y3 Canada
jknoodles.com
(905) 764-5922
https://www.yelp.com//biz/so-good-chinese-restaurant-richmond-hill-2?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=30
Restaurants
So Good Chinese Restaurant
10610 Bayview Avenue Unit 2-3 Richmond Hill, ON L4C 3N8 Canada
No link text found.
(905) 737-1628
https://www.yelp.com//biz/amys-fish-and-chips-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=30
Restaurants
Amy's Fish & Chips
1285 Elgin Mills Road E Unit 24 Richmond Hill, ON L4S 0B5 Canada
No link text found.
(905) 237-8082
https://www.yelp.com//biz/bap-zip-richmond

https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=10
Shopping
CF Markville
5000 Highway 7 E Markham, ON L3R 4M9 Canada
markville.ca/en/pages/default.…
(905) 477-6600
https://www.yelp.com//biz/heroes-world-richmond-hill?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=10
Shopping
Heroes World
9078 Leslie Street Unit 1 Richmond Hill, ON L4B 3L8 Canada
heroesworld.ca
(905) 948-1949
https://www.yelp.com//biz/platos-closet-markham?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=10
Shopping
Plato's Closet
1210 Castlemore Avenue Unit 7 Markham, ON L6E 0H7 Canada
platoscloset.com
(905) 201-5050
https://www.yelp.com//biz/oomomo-markham?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=10
Shopping
Oomomo
3265 Highway 7 E Markham, ON L3R 3P9 Canada
oomomostore.com
(905) 604-8390
https://www.yelp.com//biz/main-street-markham-markham?osq=Shopping
http

https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=40
Shopping
Legs Plus & Bra Boutique
5867 Leslie Street North York, ON M2H 1J8 Canada
braboutique.com
(416) 497-2350
https://www.yelp.com//biz/chefs-depot-scarborough?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=40
Shopping
Chef's Depot
5590 Finch Ave E Scarborough, ON M1B 1T1 Canada
chefsdepot.ca
(416) 613-8889
https://www.yelp.com//biz/zara-markham-2?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=40
Shopping
Zara
5000 Highway 7 E Markham, ON L3R 4M9 Canada
zara.com
(905) 944-1383
https://www.yelp.com//biz/main-street-unionville-markham?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=40
Shopping
Main Street Unionville
157 Main Street Unit 101B Markham, ON L3R 2G8 Canada
unionvilleinfo.com
(905) 477-0117
https://www.yelp.com//biz/the-galleria-toronto?osq=Shopping
https://www.yelp.co

159 York Boulevard Richmond Hill, ON L4B 3B4 Canada
moxies.com
(905) 764-9500
https://www.yelp.com//biz/vip-pool-and-bar-snooker-club-toronto?osq=Nightlife
https://www.yelp.com//search?find_desc=Nightlife&find_loc=Richmond+Hill&start=20
Nightlife
VIP Pool & Bar Snooker Club
301 Ellesmere Road Toronto, ON M1R 4E4 Canada
vippoolbar.com
(416) 720-0001
https://www.yelp.com//biz/harmony-club-toronto?osq=Nightlife
https://www.yelp.com//search?find_desc=Nightlife&find_loc=Richmond+Hill&start=20
Nightlife
Harmony Club
Verified by Business 
harmonyclub.ca
(416) 510-3352
https://www.yelp.com//biz/sahara-richmond-hill?osq=Nightlife
https://www.yelp.com//search?find_desc=Nightlife&find_loc=Richmond+Hill&start=20
Nightlife
Sahara
120 Newkirk Rd Richmond Hill, ON L4C 9S7 Canada
No link text found.
No phone number found.
https://www.yelp.com//biz/i-darts-edge-markham?osq=Nightlife
https://www.yelp.com//search?find_desc=Nightlife&find_loc=Richmond+Hill&start=20
Nightlife
I Darts Edge
505 Highway 7 E S

In [6]:
df = pd.DataFrame(dic)
df.to_csv('yelp_data_4cat.csv')